Truy cập để lấy LINK CỬA HÀNG

In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# Đường dẫn file Excel đầu vào chứa cột "Links"
input_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/category_2.xlsx"

# Đọc file Excel bằng pandas
df = pd.read_excel(input_file)
# Giả sử file Excel có cột "Links" chứa các URL cần truy cập
links = df["Links"].tolist()

# Sử dụng set để lưu các seller link không trùng lặp
seller_links_set = set()

# Khởi tạo Selenium WebDriver (ở đây dùng Chrome)
driver = webdriver.Chrome()
driver.maximize_window()  # Phóng to cửa sổ trình duyệt
driver.implicitly_wait(10)  # Chờ tối đa 10 giây cho các phần tử xuất hiện

# Duyệt qua từng URL trong danh sách
for url in links:
    try:
        print(f"Đang truy cập: {url}")
        driver.get(url)
        # Cho trang load dữ liệu (tùy chỉnh thời gian chờ nếu cần)
        time.sleep(2)
        
        # Tìm phần tử chứa seller link theo cấu trúc HTML được cung cấp.
        # Ta có thể sử dụng CSS selector. Trong HTML mẫu, seller link nằm trong thẻ <a> bên trong div có class:
        # "SellerName__SellerNameStyled-sc-5d1cxl-0 myqYs"
        seller_element = driver.find_element(By.CSS_SELECTOR, "div.SellerName__SellerNameStyled-sc-5d1cxl-0.myqYs a")
        
        # Lấy thuộc tính href của thẻ <a>
        seller_href = seller_element.get_attribute("href")
        print(f"  => Seller link: {seller_href}")
        
        # Thêm vào set để tránh trùng lặp
        seller_links_set.add(seller_href)
    except NoSuchElementException:
        # Nếu không tìm thấy phần tử chứa seller link
        print(f"  => Không tìm thấy seller link trên trang: {url}")
    except Exception as e:
        print(f"  => Lỗi khi xử lý {url}: {e}")

# Đóng trình duyệt
driver.quit()

# Chuyển set thành danh sách để lưu vào file Excel
seller_links_list = list(seller_links_set)

# Lưu kết quả thu thập được vào file Excel mới
output_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/store_seller_links.xlsx"
output_df = pd.DataFrame({"Seller Links": seller_links_list})
output_df.to_excel(output_file, index=False)

print(f"\nĐã lưu danh sách seller link vào file: {output_file}")


Truy cập Links cửa hàng để lấy thông tin sản phẩm.

In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# Đường dẫn file chứa danh sách cửa hàng
input_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/store_seller_links.xlsx"

# Đọc danh sách seller links
df = pd.read_excel(input_file)
seller_links = df["Seller Links"].dropna().tolist()

# Khởi tạo trình duyệt
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)  # Chờ tối đa 10 giây

# Danh sách lưu thông tin
data = []

# Duyệt từng cửa hàng
for link in seller_links:
    try:
        print(f"Đang truy cập: {link}")
        driver.get(link)
        time.sleep(3)  # Chờ trang tải

        # Lấy thông tin cửa hàng
        try:
            # Lấy danh sách các phần tử chứa thông tin cửa hàng
            store_info_values = driver.find_elements(By.CSS_SELECTOR, "span.StoreInfo__InfoValue-sc-1un24du-5.GxEKI")

            # Gán giá trị từng thông tin theo đúng thứ tự xuất hiện
            year = store_info_values[0].text if len(store_info_values) > 0 else None
            total_products = store_info_values[1].text if len(store_info_values) > 1 else None
            rating = store_info_values[2].text if len(store_info_values) > 2 else None
            review_count = driver.find_element(By.CSS_SELECTOR, "span.StoreInfo__InfoLabel-sc-1un24du-4.jpeWpY[style='margin-bottom: 2px;']").text
            followers = store_info_values[3].text if len(store_info_values) > 3 else None
            chat_response = store_info_values[4].text if len(store_info_values) > 4 else None

        except NoSuchElementException:
            year, total_products, rating, review_count, followers, chat_response = (None, None, None, None, None, None)

        # Lấy thông tin từng sản phẩm (Tên, Giá, Số lượng bán)
        try:
            product_elements = driver.find_elements(By.CSS_SELECTOR, "div.ProductRow__Container-sc-13ncpqb-1 a.Product__Wrapper-sc-n99tp2-2")
            for product in product_elements:
                try:
                    product_name = product.find_element(By.CSS_SELECTOR, "h3").text
                except NoSuchElementException:
                    product_name = None

                try:
                    sold = product.find_element(By.CSS_SELECTOR, "span.percent").text.replace("Đã bán ", "")
                except NoSuchElementException:
                    sold = None

                try:
                    price = product.find_element(By.CSS_SELECTOR, "div.Product__PriceDiscountValue-sc-n99tp2-9 span").text
                except NoSuchElementException:
                    price = None

                # Lưu từng sản phẩm vào danh sách (1 sản phẩm = 1 dòng)
                data.append({
                    "Seller Link": link,
                    "Year Joined": year,
                    "Total Products": total_products,
                    "Rating": rating,
                    "Followers": followers,
                    "Chat Response Rate": chat_response,
                    "Product Name": product_name,
                    "Price (VND)": price,
                    "Sold Quantity": sold
                })

        except NoSuchElementException:
            print(f"Không tìm thấy thông tin sản phẩm tại {link}")
            continue

    except Exception as e:
        print(f"Lỗi khi xử lý {link}: {e}")

# Đóng trình duyệt
driver.quit()

# Lưu vào Excel
output_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/store_products.xlsx"
output_df = pd.DataFrame(data)
output_df.to_excel(output_file, index=False)

print(f"\nĐã lưu thông tin vào file: {output_file}")


Đang truy cập: https://tiki.vn/cua-hang/vien-thong-ha-nhung?source_screen=product_detail&source_engine=organic
